In [1]:
# CFV Extrate                                 \\   Finish 

# Pre process
    #  1) delete all return statements then add it in the end of function

# Select Vulnerability Location               \\   Finish
    #  1) change the allocate weight method   \\   Finish
    #  2) allocate the magic number weight    \\   Finish
    #  3) add one more parameter to the function
    #     to make the path unique             \\   Finish

# Dataflow Marker                             \\   Finish 
    #  1) change the definition parameters    \\   Finish  
    
# Vulnerability Insertion                     \\   Finish
    #  1) use the track to insert vul code    \\   Finish
      
# Function Call & Controller Insertion        \\   Finish
    #  1) checksum parameter controller       \\   Finish    
    #  2) magic number length parameter       \\   Finish    
    #  3) change input type to "scanf"        \\   Finish  
    #  4）change the comparation expression   \\   Finish
    
# Key Words Statements Insertion              \\   Finish
    #  1) change the loop and if statements   \\   Finish
    #  2) fix the bug                         \\   Finish 
    #  3) add the magic parameter controller  \\   Finish
    #  4）add function call                   \\   Finish
    #  5) more key words processor            \\   Finish
    #  6) add if numbers controller parameter \\   Finish
    #  7) remove some special symbols         \\   Finish 
    
# Environment Code Generation                 \\   Finish
    #  1) add unique function call processor  \\   Finish   
    
# Formalization                               \\   Finish
    #  1) add includings                      \\   Finish
    #  2) add vul lib includings              \\   Finish

# Known Bug:
#    1) the optional question in environment code generation
#    2) the function calls may be inserted in front of the dataflow     (Fixed)

# TO DO :
#    1) add fake vulberability 

In [2]:
import os
from xml.dom.minidom import parse
import xml.dom.minidom
import numpy as np
import random
import csv
import networkx as nx
import unicodedata
import copy

import CFV_Extractor as CE
import Select_Vul_Location as VL
import FuncCall_Controller_Insertion as FCI
import Vul_Marker as VM
import Vul_Insertion as VI
import Environment_Code_Generator as ECG
import Environment_Key_Statement_Insertion as EKSI
import Formalization as Formal


GrammarTree = r"./GrammarTree.xml" # the grammar tree path
VulLib = r"./VulnerabilityLib.xml" # the vulnerability lib path

FilePath = "./gpickle/AVR-Keyboard_0.gpickle"

In [3]:
# ================ ulit parameters =======================

# the unique path check number
unique_magic_number = random.randint(0,1000)

# the chose vulnerability name
Vul_name = "CWE-761"

# dataflow parameters and environment parameters

# ================ dataflow parameters =====================

# the input number indicte the deepth of the vulnerability
# e.g. ' input_number = 10 ' means the vulnerability was hiden in 
#       a 10 deepth input nest (if you want to trige the vulnerability,
#       you have to pass the 10 if-input-checks)
#  the input check looks like ' if( strcmp(controller_1,"abcd") > 0)'
dataflow_parameter_input_number = 3            
            
# the percentage of magic number based on input numbers
# e.g. ' magic_number = 0.1 ' the magic number takes 10% of input numbers
#       if the input_number equals 10, one of them (10%) will become a 
#       magic number check
#  the magic number check looks like ' if( strcmp(controller_1,"abcd") == 0) ' 
dataflow_parameter_magic_number = 0           
            
            
# the magic_number_mix and max limit the input length
# for e.g. ' min = 1 && max = 5 ' means the length of the magic number check will be randonly picked from 1 to 5
dataflow_parameter_magic_number_length_min = 1
dataflow_parameter_magic_number_length_max = 5

# the type number of checksum
# 1 = remainder_check
# 2 = .....(to be continued)
# the checksum check look like ' if(remainder_check(controller_1,checksum_div,length_of_controller_1) '
# more detail about the checksum function , please see 'Formalization.py'
dataflow_parameter_checksum_type = 1

# the dividend for remainder_checksum
dataflow_parameter_checksum_div = 100

# the percentage of checksum number based on magic numbers
# e.g. ' checksum = 0.1 ' the checksum takes 10% of magic numbers
#       if the input_number equals 10, one of them (10%) will become a 
#       checksum check
dataflow_parameter_checksum = 0

# ================ environment parameters =====================

# branches means the input check numbers in environment code
# e.g. ' branches = 100 ' means there are 100 input checks in the environment code
environment_parameter_branches = 3

# unuseful parameter ---
# need to remove loop_times_min and max
environment_parameter_loop_times_min = 1
environment_parameter_loop_times_max = 10

environment_parameter_magic_number_length_min = 1  # same meaning with dataflow one but in environment
environment_parameter_magic_number_length_max = 5  # same meaning with dataflow one but in environment
environment_parameter_magic_number = 0             # same meaning with dataflow one but in environment

environment_parameter_checksum_type = 1     # same meaning with dataflow one but in environment
environment_parameter_checksum_div = 100    # same meaning with dataflow one but in environment
environment_parameter_checksum_number = 0   # same meaning with dataflow one but in environment

# control loop time of all the loopers in environment code
environment_paramenter_loop_time = 5


In [4]:
# abandoned function
# to test the function callee and caller`s relationship in their refids and refedids
def recover_test():
    for i in range(1,len(selectedPath)):
        if nodeSet[selectedPath[i]]["id"] in nodeSet[selectedPath[i-1]]["refids"]:
            print("1")
        else:
            print("2")

    for i in range(1,len(selectedPath)):
        if nodeSet[selectedPath[i-1]]["id"] in nodeSet[selectedPath[i]]["refedids"]:
            print("3")
        else:
            print("4")

In [5]:
# test-use only
# display the vulnerablity related data:
#     1) vul_path
#     2) nest_path_weight
#     3) maigc_number_weight
#     4) checksum_weight
def display_vul_path():
    for i in range(0,len(selectedPath)):
        print(nodeSet[selectedPath[i]]["name"])
        print(nodeSet[selectedPath[i]]["cfv"])
        print("==================================")
    print("vul_path:")
    print(selectedPath)
    print("path_weight:")
    print(selectedWeight)
    print("magic number weight")
    print(magicWeight)
    print("checksum weight")
    print(checksumWeight)

In [6]:
# get all hyper parameters and organize them in a txt content format
def getParameters():
    
    content = ""
    
    # ulit parameters
    content += "\n\n================= ulit parameters ======================\n\n"
    content += "unique_magic_number = "+ str(unique_magic_number)+"\n"
    content += "vulnerability_name = "+ str(Vul_name)+"\n"
    content += "\n================= ulit parameters ends ======================"
    
    # dataflow parameters
    content +="\n\n================= dataflow parameters ======================\n\n"
    content +="dataflow_parameter_input_number = "+str(dataflow_parameter_input_number)+"\n\n"
    content +="dataflow_parameter_magic_number = "+str(dataflow_parameter_magic_number)+"\n"
    content +="dataflow_parameter_magic_number_length_min = "+str(dataflow_parameter_magic_number_length_min)+"\n"
    content +="dataflow_parameter_magic_number_length_max = "+str(dataflow_parameter_magic_number_length_max)+"\n\n"
    content +="dataflow_parameter_checksum_type = "+str(dataflow_parameter_checksum_type)+"\n"
    content +="dataflow_parameter_checksum_div = "+str(dataflow_parameter_checksum_div)+"\n"
    content +="dataflow_parameter_checksum = "+str(dataflow_parameter_checksum)+"\n"
    content +="\n================= dataflow parameters ends ======================"

    # environment parameters
    content +="\n\n================= environment parameters ======================\n\n"
    content +="environment_parameter_branches = "+str(environment_parameter_branches)+"\n\n"
    content +="environment_parameter_loop_times_min = "+str(environment_parameter_loop_times_min)+"\n"
    content +="environment_parameter_loop_times_max = "+str(environment_parameter_loop_times_max)+"\n\n"
    content +="environment_parameter_magic_number_length_min = "+str(environment_parameter_magic_number_length_min)+"\n"
    content +="environment_parameter_magic_number_length_max = "+str(environment_parameter_magic_number_length_max)+"\n"
    content +="environment_parameter_magic_number = "+str(environment_parameter_magic_number)+"\n\n"
    content +="environment_parameter_checksum_type = "+str(environment_parameter_checksum_type)+"\n"
    content +="environment_parameter_checksum_div = "+str(environment_parameter_checksum_div)+"\n"
    content +="environment_parameter_checksum_number = "+str(environment_parameter_checksum_number)+"\n\n"
    content +="environment_paramenter_loop_time = "+str(environment_paramenter_loop_time)+"\n"
    content +="\n================= environment parameters ends ======================"
    
    return content

In [7]:
# filter for the word list
# filter all keyword in 'filter_list'
# parameter: nodeSet -> which contains all the function node
def pre_process_filter(nodeSet):
    filter_list = ["switch","case","?:","break","default","continue"]
    for i in range(0,len(nodeSet)):
        index = 0
        while index<len(nodeSet[i]["cfv"]):
            if nodeSet[i]["cfv"][index] in filter_list:
                del nodeSet[i]["cfv"][index]
            else:
                index+=1
        
    return nodeSet
                
# rename the confilcted function name
# add a prefix in the confilcted name and change the name in all reference or referenced call
# parameter: nodeSet -> which contains all the function node
def pre_process_func_name(nodeSet):
    func_name_list = []
    for i in range(0,len(nodeSet)):
        if nodeSet[i]["name"] not in func_name_list:
            func_name_list.append(nodeSet[i]["name"])
        else:
            new_name = "new_"+nodeSet[i]["name"]
            old_name = nodeSet[i]["name"]
            retType = nodeSet[i]["full_definition"].split(nodeSet[i]["name"])[0].strip()
            
            nodeSet[i]["full_definition"] = nodeSet[i]["full_definition"].replace(old_name,new_name)
            nodeSet[i]["name"] = new_name 
            
            # replace the old name in refDef of other nodes with new name
            for index in range(0,len(nodeSet)):
                for j in range(0,len(nodeSet[index]["refDef"])):
                    if nodeSet[index]["refDef"][j][0] == old_name:
                        if nodeSet[index]["refDef"][j][1].split(old_name)[0].strip()==retType:
                            refDef = (new_name,nodeSet[index]["refDef"][j][1].replace(old_name,new_name))
                            nodeSet[index]["refDef"][j] = refDef
                            
    return nodeSet

# add return to the blank cfv node
# if the blank cfv node needs a return variable, the cfv turns to ['{','return','}']
# if the blank cfv node doesn`t have a return variable, the cfv turns to ['{',';','}']
# parameter: nodeSet -> which contains all the function node
def pro_process_blank_cfv(nodeSet):
    for i in range(0,len(nodeSet)):
        if len(nodeSet[i]["cfv"])==0:
            #print(nodeSet[i]["full_definition"])
            if nodeSet[i]["full_definition"].startswith("void"):
                nodeSet[i]["cfv"] = ['{',';','}']
            else:
                nodeSet[i]["cfv"] = ['{','return','}']
    return nodeSet

In [8]:
# get the node list from gpickle file 
extractor = CE.CFV_Extractor(FilePath)
nodeSet = extractor.getAllCFV()

In [9]:
nodeSet = pre_process_filter(nodeSet)  # pre process for filtering some keywords
nodeSet = pre_process_func_name(nodeSet) # pre process for renameing the confilcted functions
nodeSet = pro_process_blank_cfv(nodeSet) # pre process for adding content into the blank cfv

In [10]:
for i in range(0,len(nodeSet)):
    print(nodeSet[i]["name"])
    print(nodeSet[i]["cfv"])

main
['{', ';', ';', ';', 'for', '{', ';', '}', 'for', '{', 'for', '{', ';', ';', 'for', '{', 'if', '{', ';', '}', ';', 'if', '{', 'if', '{', ';', '}', 'else', '{', ';', '}', '}', 'else if', '{', ';', '}', ';', '}', ';', '}', '}', '}']
init
['{', ';', ';', ';', ';', ';', ';', 'while', '{', ';', '}', ';', ';', ';', ';', 'for', '{', ';', ';', '}', ';', '}']
usb_init
['{', ';', ';', ';', 'while', '{', ';', '}', ';', ';', ';', ';', ';', '}']
usb_configured
['{', 'return', '}']
setup_io_pins
['{', ';', ';', ';', 'for', '{', ';', ';', '}', '}']
setup_leds
['{', ';', ';', '}']
setup_bounce_timer
['{', ';', ';', ';', ';', '}']
pull_column
['{', ';', '}']
key_press
['{', ';', ';', ';', 'if', '{', ';', '}', 'else', '{', 'for', '{', ';', '}', ';', '}', ';', '}']
send
['{', ';', 'for', '{', ';', '}', ';', ';', '}']
usb_keyboard_send
['{', ';', 'if', '{', 'return', '}', ';', ';', ';', ';', 'while', '{', 'if', '{', '}', ';', 'if', '{', 'return', '}', 'if', '{', 'return', '}', ';', ';', ';', '}', ';'

In [11]:
# =================================== locate the vul location ===================================

In [12]:
# via the customized hyper parameter, find a suitable place for vulnerability
locator = VL.Select_Vul_Location(extractor                         # the cfv extrotor
                                 ,dataflow_parameter_input_number  # the hyper parameter - input number
                                 ,dataflow_parameter_magic_number  # the hyper parameter - magic number
                                 ,dataflow_parameter_checksum)     # the hyper parameter - checksum

# the vul_path

# e.g. vul_path = [0,10,11,12]
#      it means that to trige the vulnerability, you have to pass the No.0 node, No.10 node and No.12 node
selectedPath = locator.getSelectedPath()

# the nest_weight
# it should be used with vul_path and the index of the list indicate the index in vul_path

# e.g. nest_weight = [2,1,0,1] && vul_path = [0,10,11,12]

# the nest_weight[0] = 2 means in the No.0 node (vul_path[0]), we call the No.10 node (vul_path[1]) function in a 
#   two-level (nest_weight[0] = 2 ) if nest.
# the nest_weight[1] = 1 means in the No.10 node (vul_path[1]), we call the No.11 node (vul_path[2]) function in a 
#   one-level (nest_weight[1] = 1 ) if nest.
#  ````````
# the nest_weight[3] = 1 means in the No.12 node (vul_path[1]), we trige the vulnerability in a one-level 
#   (nest_weight[3] = 1) if nest
selectedWeight = locator.getSelectedWeight()

# the magic_number_weight
# it was conducted by hyper parameter maigc number and the nest_weight

# e.g. magic_number_weight = [1,1,0,0] && nest_weight = [2,1,0,1]
# the magic number weight is based on the nest_weight
# magic_number_weight[0] = 1 means in the 2 input checks (nest_weight[0]=2), the 1 of them will become magic number check
magicWeight = locator.getMagicNumberWeight()

# the checksum_weight
# it was conducted by hyper parameter checksum and the nest_weight

# e.g. checksum_weight = [1,1,0,0] && magic_number_weight = [2,1,0,1]
# the checksum_weight is based on the magic_number_weight
# checksum_weight[0] = 1 means in the 2 magic number check (magic_number_weight[0]=2), the 1 of them will become checksum
checksumWeight = locator.getChecksumWeight()

In [13]:
locator.displayOutput()

dataflow_input:
3
path:
[0, 8]
cfvWeight:
[2, 1]
weight:
[2, 1]
magic number weight:
[0, 0]
checksum weight:
[0, 0]


In [14]:
# =================================== locate the vul location Ends ===================================

In [15]:
for i in range(0,len(nodeSet)):
    print(nodeSet[i]["name"])
    print(nodeSet[i]["cfv"])

main
['{', ';', ';', ';', 'for', '{', ';', '}', 'for', '{', 'for', '{', ';', ';', 'for', '{', 'if', '{', ';', '}', ';', 'if', '{', 'if', '{', ';', '}', 'else', '{', ';', '}', '}', 'else if', '{', ';', '}', ';', '}', ';', '}', '}', '}']
init
['{', ';', ';', ';', ';', ';', ';', 'while', '{', ';', '}', ';', ';', ';', ';', 'for', '{', ';', ';', '}', ';', '}']
usb_init
['{', ';', ';', ';', 'while', '{', ';', '}', ';', ';', ';', ';', ';', '}']
usb_configured
['{', 'return', '}']
setup_io_pins
['{', ';', ';', ';', 'for', '{', ';', ';', '}', '}']
setup_leds
['{', ';', ';', '}']
setup_bounce_timer
['{', ';', ';', ';', ';', '}']
pull_column
['{', ';', '}']
key_press
['{', ';', ';', ';', 'if', '{', ';', '}', 'else', '{', 'for', '{', ';', '}', ';', '}', ';', '}']
send
['{', ';', 'for', '{', ';', '}', ';', ';', '}']
usb_keyboard_send
['{', ';', 'if', '{', 'return', '}', ';', ';', ';', ';', 'while', '{', 'if', '{', '}', ';', 'if', '{', 'return', '}', 'if', '{', 'return', '}', ';', ';', ';', '}', ';'

In [16]:
# =================================== vul location marker ===================================

In [17]:
# add the dataflow marks into the node list
nodeSet = VM.markDataflow(nodeSet,selectedWeight,selectedPath) 

In [18]:
# change the definition of functions which in dataflow
# add a new parameter named 'unique_parameter' to control that there is only one way to trige the vulnerability
nodeSet = VM.changeDefinitions(nodeSet,selectedPath)

In [19]:
# =================================== vul location marker Ends ===================================

In [20]:
for i in range(0,len(nodeSet)):
    print(nodeSet[i]["name"])
    print(nodeSet[i]["cfv"])

main
['{', ';', ';', ';', 'for', '{', ';', '}', 'for', '{', 'for', '{', ';', ';', 'for', '{', 'if', '{', ';', '}', ';', 'if', '{', 'if', '{', 'Path/key_press/8', ';', '}', 'else', '{', ';', '}', '}', 'else if', '{', ';', '}', ';', '}', ';', '}', '}', '}']
init
['{', ';', ';', ';', ';', ';', ';', 'while', '{', ';', '}', ';', ';', ';', ';', 'for', '{', ';', ';', '}', ';', '}']
usb_init
['{', ';', ';', ';', 'while', '{', ';', '}', ';', ';', ';', ';', ';', '}']
usb_configured
['{', 'return', '}']
setup_io_pins
['{', ';', ';', ';', 'for', '{', ';', ';', '}', '}']
setup_leds
['{', ';', ';', '}']
setup_bounce_timer
['{', ';', ';', ';', ';', '}']
pull_column
['{', ';', '}']
key_press
['{', ';', ';', ';', 'if', '{', 'Vul/KeyStatement', ';', '}', 'else', '{', 'for', '{', ';', '}', ';', '}', ';', '}']
send
['{', ';', 'for', '{', ';', '}', ';', ';', '}']
usb_keyboard_send
['{', ';', 'if', '{', 'return', '}', ';', ';', ';', ';', 'while', '{', 'if', '{', '}', ';', 'if', '{', 'return', '}', 'if', '{'

In [21]:
# =================================== vul insertion ===================================

In [22]:
# get the vul statement from vul lib xml file
vul_node = VI.getVulNode(Vul_name,VulLib)
# get the variable definition from the xml node
vul_definition = VI.getVulDefinition(vul_node) 
# get the vulnerability statements from the xml node
vul_statement,key_index = VI.getVulStatement(vul_node)

In [23]:
# get the vul track via the marked node and vulnerability statement
vul_track = VI.getVulTrack(nodeSet[selectedPath[-1]]["cfv"],  # the marked node (the place to insert the vulnerability)
                           vul_statement, # vulnerability satatement 
                           key_index)     # indicate the key line in statements 

In [24]:
# use the track to insert the vul code
nodeSet[selectedPath[-1]]["cfv"] = VI.insertStatementByTrack(vul_track,
                                                            nodeSet[selectedPath[-1]]["cfv"],
                                                            unique_magic_number)
# insert the variable definition in the node
nodeSet[selectedPath[-1]]["cfv"] = VI.insertDefinitions(nodeSet[selectedPath[-1]]["cfv"],
                                                       vul_definition)

In [25]:
# =================================== vul insertion Ends ===================================

In [26]:
for i in range(0,len(nodeSet)):
    print(nodeSet[i]["name"])
    print(nodeSet[i]["cfv"])

main
['{', ';', ';', ';', 'for', '{', ';', '}', 'for', '{', 'for', '{', ';', ';', 'for', '{', 'if', '{', ';', '}', ';', 'if', '{', 'if', '{', 'Path/key_press/8', ';', '}', 'else', '{', ';', '}', '}', 'else if', '{', ';', '}', ';', '}', ';', '}', '}', '}']
init
['{', ';', ';', ';', ';', ';', ';', 'while', '{', ';', '}', ';', ';', ';', ';', 'for', '{', ';', ';', '}', ';', '}']
usb_init
['{', ';', ';', ';', 'while', '{', ';', '}', ';', ';', ';', ';', ';', '}']
usb_configured
['{', 'return', '}']
setup_io_pins
['{', ';', ';', ';', 'for', '{', ';', ';', '}', '}']
setup_leds
['{', ';', ';', '}']
setup_bounce_timer
['{', ';', ';', ';', ';', '}']
pull_column
['{', ';', '}']
key_press
['{', 'char * vul_var;', 'vul_var=(char*)malloc(20*sizeof(char));', ';', ';', ';', 'if', '{', 'strcpy(vul_var, "CWE-761");', 'if(uni_para == 592)', '{', 'vul_var = vul_var + 1;', '}', 'free(vul_var);', 'Vul/KeyStatement', ';', '}', 'else', '{', 'for', '{', ';', '}', ';', '}', ';', '}']
send
['{', ';', 'for', '{', 

In [27]:
# =================================== insert function call and controller ===================================

In [28]:
# insert the function call
nodeSet = FCI.insertFunctionCall(nodeSet,selectedPath,unique_magic_number)

In [29]:
number = 0
for index in selectedPath:
    position = FCI.locateController(nodeSet[index]["cfv"])
    magic_number_list = [0]*selectedWeight[number]
    
    temp = magicWeight[number]
    while temp != 0:
        index_magic = random.randint(0,selectedWeight[number]-1)
        if  magic_number_list[index_magic] == 0:
            temp-=1
            magic_number_list[index_magic]+=1
            
    temp = checksumWeight[number]
    while temp != 0:
        index_magic = random.randint(0,selectedWeight[number]-1)
        if  magic_number_list[index_magic] == 1:
            temp-=1
            magic_number_list[index_magic]+=1
    
    
    nodeSet[index] = FCI.insertController(nodeSet[index],
                                          position,
                                          dataflow_parameter_magic_number_length_min,
                                          dataflow_parameter_magic_number_length_max,
                                          magic_number_list,
                                          dataflow_parameter_checksum_type,
                                           dataflow_parameter_checksum_div)
    nodeSet[index]["cfv"] = FCI.uselessIfInsertion(nodeSet[index]["cfv"])
    number+=1

In [30]:
# =================================== insert function call and controller Ends ===================================

In [31]:
for i in range(0,len(nodeSet)):
    print(nodeSet[i]["name"])
    print(nodeSet[i]["cfv"])

main
['{', ';', ';', ';', 'for', '{', ';', '}', 'for/1', '{', 'for/1', '{', ';', ';', 'for/1', '{', 'if(1)', '{', ';', '}', ';', 'char controller4vul_1[1];', 'fgets(controller4vul_1 ,2 ,stdin);', 'if( strcmp( controller4vul_1, "M") < 0)', '{', 'char controller4vul_2[5];', 'fgets(controller4vul_2 ,6 ,stdin);', 'if( strcmp( controller4vul_2, "H%q@R") < 0)', '{', '#void  = key_press(#char,#controller4unique)', ';', '}', 'else', '{', ';', '}', '}', 'else if', '{', ';', '}', ';', '}', ';', '}', '}', '}']
init
['{', ';', ';', ';', ';', ';', ';', 'while', '{', ';', '}', ';', ';', ';', ';', 'for', '{', ';', ';', '}', ';', '}']
usb_init
['{', ';', ';', ';', 'while', '{', ';', '}', ';', ';', ';', ';', ';', '}']
usb_configured
['{', 'return', '}']
setup_io_pins
['{', ';', ';', ';', 'for', '{', ';', ';', '}', '}']
setup_leds
['{', ';', ';', '}']
setup_bounce_timer
['{', ';', ';', ';', ';', '}']
pull_column
['{', ';', '}']
key_press
['{', 'char * vul_var;', 'vul_var=(char*)malloc(20*sizeof(char));'

In [32]:
# =================================== environment key statement insertion generation ===================================

In [33]:
def scanIFinEnvironment(nodeSet):
    node_if_numbers = [0] *  len(nodeSet)
    for index in range(0,len(nodeSet)):
        cfv = nodeSet[index]["cfv"]
        for i in range(0,len(cfv)):
            if cfv[i] == "if" or cfv[i] == "else if":
                node_if_numbers[index] += 1
    return node_if_numbers

In [34]:
def allocateNumbers(number_lsit,limit_times):
    result_list = [0] * len(number_lsit)
    temp = limit_times
    while temp != 0:
        index = random.randint(0,len(number_lsit)-1)
        if  result_list[index] < number_lsit[index]:
            temp-=1
            result_list[index]+=1
    
    return result_list


def changeNumber2List(source_list,target_list):
    result_list = []
    for i in range(0,len(source_list)):
        temp_list = [0] * source_list[i]
        limit = target_list[i]
        while limit != 0:
            index = random.randint(0,source_list[i]-1)
            if temp_list[index] == 0:
                limit-=1
                temp_list[index]+=1
        result_list.append(temp_list)
    return result_list


def recordBranchIndex(branch_list,level):
    result_list = []
    for i in range(0,len(branch_list)):
        for j in range(0,len(branch_list[i])):
            if branch_list[i][j]==level:
                result_list.append([i,j])
    return result_list

def addNumberIntoBranch(branch_list,level,limit,record):
    result_list = copy.copy(branch_list)
    temp_list = [level] * len(record)
    
    temp = limit
    while temp!=0:
        index = random.randint(0,len(temp_list)-1)
        if temp_list[index]==level:
            temp-=1
            temp_list[index]+=1
    
    for index in range(0,len(record)):
        result_list[record[index][0]][record[index][1]] = temp_list[index]
    
    return result_list

In [35]:
node_if_numbers = scanIFinEnvironment(nodeSet)

branch_list = allocateNumbers(node_if_numbers,environment_parameter_branches)

branch_list = changeNumber2List(node_if_numbers,branch_list)

record4magic_number = recordBranchIndex(branch_list,1)

branch_list = addNumberIntoBranch(branch_list,1,environment_parameter_magic_number,record4magic_number)

record4checksum = recordBranchIndex(branch_list,2)

branch_list = addNumberIntoBranch(branch_list,2,environment_parameter_checksum_number,record4checksum)

In [36]:
for index in range(0,len(nodeSet)):
    func_flag = 1
    if index in selectedPath:
        func_flag = 0
    statementInserter = EKSI.StatementInserter(nodeSet[index],
                                          environment_parameter_loop_times_min,
                                          environment_parameter_loop_times_max,
                                          branch_list[index],
                                          environment_parameter_magic_number_length_min,
                                          environment_parameter_magic_number_length_max,
                                          environment_parameter_checksum_type,
                                          environment_parameter_checksum_div,
                                          func_flag)
    nodeSet[index]["cfv"] = statementInserter.getCFV()
    del statementInserter

In [37]:
# =================================== environment key statement insertion generation Ends ===================================

In [38]:
for i in range(0,len(nodeSet)):
    print(nodeSet[i]["name"])
    print(nodeSet[i]["cfv"])

main
['{', ';', ';', ';', 'for(int looper_1=0; looper_1<3;looper_1++)', '{', ';', '}', 'for(int looper_2=0; looper_2<1;looper_2++)', '{', 'for(int looper_3=0; looper_3<1;looper_3++)', '{', ';', ';', 'for(int looper_4=0; looper_4<1;looper_4++)', '{', 'if(1)', '{', ';', '}', ';', 'char controller4vul_1[1];', 'fgets(controller4vul_1 ,2 ,stdin);', 'if( strcmp( controller4vul_1, "M") < 0)', '{', 'char controller4vul_2[5];', 'fgets(controller4vul_2 ,6 ,stdin);', 'if( strcmp( controller4vul_2, "H%q@R") < 0)', '{', '#void  = key_press(#char,#controller4unique)', ';', '}', 'else', '{', ';', '}', '}', 'char controller_1[5];', 'fgets(controller_1 ,5 ,stdin);', 'if( strcmp( controller_1, "33K5") > 0)', '{', ';', '}', ';', '}', ';', '}', '}', '}']
init
['{', '#setup_leds(#void)', ';', ';', ';', ';', ';', ';', 'int looper_1 = 0;', 'while(looper_1 < 6)', '{', 'looper_1 += 1;', '#setup_bounce_timer(#void)', ';', '}', ';', '#usb_init(#void)', ';', '#unsigned int = usb_configured(#void)', '#setup_io_pin

In [39]:
# =================================== environment code generation ===================================

In [40]:
for index in range(0,len(nodeSet)):
    generator = ECG.CodeGenerator(GrammarTree,nodeSet,index,selectedPath,unique_magic_number)
    nodeSet[index]["cfv"] = generator.getCFV()

['{-1-', ';-1', ';-1', ';-1', 'for(int looper_1=0; looper_1<3;looper_1++)', '{-1_1-', ';-1_1', '}', 'for(int looper_2=0; looper_2<1;looper_2++)', '{-1_2-', 'for(int looper_3=0; looper_3<1;looper_3++)', '{-1_2_1-', ';-1_2_1', ';-1_2_1', 'for(int looper_4=0; looper_4<1;looper_4++)', '{-1_2_1_1-', 'if(1)', '{-1_2_1_1_1-', ';-1_2_1_1_1', '}', ';-1_2_1_1', 'char controller4vul_1[1];', 'fgets(controller4vul_1 ,2 ,stdin);', 'if( strcmp( controller4vul_1, "M") < 0)', '{-1_2_1_1_2-', 'char controller4vul_2[5];', 'fgets(controller4vul_2 ,6 ,stdin);', 'if( strcmp( controller4vul_2, "H%q@R") < 0)', '{-1_2_1_1_2_1-', '#void  = key_press(#char,#controller4unique)', ';-1_2_1_1_2_1', '}', 'else', '{-1_2_1_1_2_2-', ';-1_2_1_1_2_2', '}', '}', 'char controller_1[5];', 'fgets(controller_1 ,5 ,stdin);', 'if( strcmp( controller_1, "33K5") > 0)', '{-1_2_1_1_3-', ';-1_2_1_1_3', '}', ';-1_2_1_1', '}', ';-1_2_1', '}', '}', '}']
['{-1-', '#setup_leds(#void)', ';-1', ';-1', ';-1', ';-1', ';-1', ';-1', 'int looper

In [41]:
# =================================== environment code generation Ends ===================================

In [42]:
# =================================== formalization ===================================

In [43]:
trackPath = ""
if "/" in FilePath:
    filename = FilePath.split("/")[-1].split(".")[0]
    trackPath = filename +".txt"
    
with open(trackPath,"w",encoding="utf-8") as f:
    f.truncate()
    
    # write the track
    track = Formal.getTrackContent(selectedPath,nodeSet)
    f.write(track)
    
    # write the parameters
    parameter_content = getParameters()
    f.write(parameter_content)


In [44]:
nodeSet[selectedPath[-1]]["cfv"] = Formal.cleanVulMark(nodeSet[selectedPath[-1]]["cfv"])
nodeSet[0]["cfv"].insert(1,"int uni_para ="+str(unique_magic_number)+";")

In [45]:
CPath = ""
if "/" in FilePath:
    filename = FilePath.split("/")[-1].split(".")[0]
    CPath = filename +".c"
    
with open(CPath,"w",encoding="utf-8") as f:
    f.truncate()
    
    header_content = Formal.header4program(GrammarTree,vul_node)
    f.write(header_content)
    
    checksum = Formal.swithForChecksum(dataflow_parameter_checksum_type)
    f.write(checksum[0])
    f.write(checksum[1])
    if dataflow_parameter_checksum_type != environment_parameter_checksum_type:
        checksum = Formal.swithForChecksum(environment_parameter_checksum_type)
        f.write(checksum[0])
        f.write(checksum[1])
    
    for index in range(len(nodeSet)-1,0,-1):
        definition = Formal.Definition2Code(nodeSet,index)+";\n"
        f.write(definition)
    for index in range(len(nodeSet)-1,-1,-1):
        content = Formal.Node2Code(nodeSet,index)
        f.write(content)

In [46]:
# =================================== formalization Ends ===================================